# Recommender system for a medical supplies company

Projenin amacı, bir tavsiye sistemi uygulayarak şirketin artan satışlar elde etmesine yardımcı olmaktır.

<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2020/09/Article-Recommender.png">

In [4]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns",100) #Satır ve sütunların hepsini göstermeye yarıyor
pd.set_option("display.max_rows",190) #isnull gibi degerlerin tamamını görmeye yarıyor

<a href="https://drive.google.com/file/d/1svnXrwmzLbqebqtDgdKhLLhUny7-eyqp/view?usp=drive_link"> Dataya Buradan Erişebilrsiniz</a>

### EDA

In [6]:
df = pd.read_csv("PBL 5 recommendation data.csv", encoding='Latin=1')

In [7]:
df.head()

,Customers.id,Customers.fname,Customers.lname,Customers.company,Customers.create_date,Customers.status,Customers.mailing,Customers.reminders,Customers.tax_exempt,Customers.account_id,Customers.sales_rep,Customers.rewards,Customers.profile_id,Customers.last_modified,Customers.customer_type,Orders.id,Orders.customer_id,Orders.fname,Orders.lname,Orders.company,Orders.order_number,Orders.reorder_id,Orders.external_source,Orders.external_id,Orders.currency,Orders.sales_rep,Orders.subtotal,Orders.tax,Orders.shipping,Orders.coupon_id,Orders.coupon_amount,Orders.gift_id,Orders.gift_amount,Orders.fee_name,Orders.fee_amount,Orders.discount_name,Orders.discount_amount,Orders.total,Orders.balance_due,Orders.shipping_carrier,Orders.shipping_method,Orders.shipping_trans,Orders.shipping_flags,Orders.weight,Orders.tracking,Orders.payment_status,Orders.payment_date,Orders.payment_user,Orders.payment_type,Orders.payment_method,...,Products.seo_description,Products.seo_keywords,Products.seo_header,Products.seo_footer,Products.seo_url,Products.seo_category,Products.unit,Products.packaging,Products.display_packaging,Products.multiple,Products.length,Products.width,Products.height,Products.rx,Products.latex,Products.upc,Products.msds_link,Products.msds_label,Products.lit_link,Products.lit_label,Products.hcpcs,Products.case_qty,Products.markup,Products.override_markup,Products.notes,Products.import_flags,Products.map_price,Products.features_title,Products.warranty,Products.hygienic,Products.default_quantity,Products.user_size,Products.assembly,Products.installation,Products.shipping_length,Products.shipping_width,Products.shipping_height,Products.shipping_weight,Products.handling_time,Products.rotation_link,Products.google_shopping_label,Products.product_option,Products.size,Products.material,Products.arm_style,Products.leg_style,Products.seat_size,Products.family_id,Products.saved_status,Products.freight_cost
0,797,Christy,Dill,Company0,1426018724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1437764306,0.0,3758,797,Christy,Dill,Company0,3758,NaN,NaN,NaN,USD,NaN,57.20,0.0,9.95,13.0,2.86,NaN,NaN,NaN,NaN,NaN,NaN,64.29,NaN,fedex,11|Ground,NaN,NaN,NaN,5.7204E+14,3.0,1.426019e+09,NaN,authorize.net,NaN,...,NaN,NaN,NaN,NaN,Basic-Rollators-For-Adults-With-Seat-Green-6-W...,0.0,EA,1 Each / Each,NaN,1 EA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E0149 +E0156,1.0,NaN,0.0,NaN,136.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,24.80,7.50,30.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF61071,0.0,NaN
1,3,John,Smith,Company1,1386089139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1437764354,0.0,23,3,John,Smith,Company1,23,NaN,NaN,NaN,USD,NaN,20.00,NaN,9.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.99,NaN,fedex,11|Ground,NaN,NaN,NaN,1.28597E+14,3.0,1.386090e+09,NaN,authorize.net,NaN,...,NaN,NaN,NaN,NaN,Urinary-Drain-Bags0000,0.0,EA,1 Each / Each,NaN,1 EA,NaN,NaN,NaN,Rx,Yes,NaN,NaN,NaN,https://medline360.medline.com:443/irj/servlet...,508f2d75-582e-2e10-2eab-d83980a9b1d0,A4357,1.0,NaN,0.0,OFF sale cause it is an RX item,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,8.00,7.00,1.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PF02132,NaN,NaN
2,3,John,Smith,Company1,1386089139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1437764354,0.0,9531,3,John,Smith,Company1,9531,NaN,NaN,NaN,USD,NaN,68.78,0.0,9.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.73,NaN,manual,0|Standard Shipping,NaN,NaN,NaN,NaN,3.0,1.449604e+09,genlee166,authorize.net,NaN,...,NaN,NaN,NaN,NaN,SensiCare-Nitrile-Exam-GlovesBlueXX-Large,0.0,CS,800 Each / Case,NaN,1 CS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO GOVERNMENT OR SUPPLIER HCPCS CODE VERIFICATION,800.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.23,10.04,10.24,NaN,NaN,NaN,NaN,NaN,2 x Extra large,Nitrile,NaN,NaN,NaN,PF00342,0.0,NaN
3,4,James,Anderson,NaN,1386780263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1437762646,0.0,29,4,James,Anderson,NaN,29,NaN,Ebay,NaN,USD,NaN,19.56,0.0,9.95,NaN,NaN,NaN,NaN,NaN,0.04,NaN,NaN,29.55,NaN,NaN,11|Ground,NaN,NaN,NaN,7.96468E+14,3.0,1.386780e+09,NaN,NaN,Credit Card,...,NaN,NaN,NaN,NaN,Basket-for-2-Butt

In [8]:
df.shape

(4194, 181)

In [9]:
df.columns

Index(['Customers.id', 'Customers.fname', 'Customers.lname',
       'Customers.company', 'Customers.create_date', 'Customers.status',
       'Customers.mailing', 'Customers.reminders', 'Customers.tax_exempt',
       'Customers.account_id',
       ...
       'Products.google_shopping_label', 'Products.product_option',
       'Products.size', 'Products.material', 'Products.arm_style',
       'Products.leg_style', 'Products.seat_size', 'Products.family_id',
       'Products.saved_status', 'Products.freight_cost'],
      dtype='object', length=181)

In [10]:
df.isnull().sum()

Customers.id                         0
Customers.fname                      0
Customers.lname                      0
Customers.company                 3467
Customers.create_date                0
Customers.status                  4093
Customers.mailing                 2080
Customers.reminders               4194
Customers.tax_exempt              4189
Customers.account_id              4191
Customers.sales_rep               4194
Customers.rewards                 4194
Customers.profile_id              4194
Customers.last_modified              0
Customers.customer_type            423
Orders.id                            0
Orders.customer_id                   0
Orders.fname                         0
Orders.lname                         0
Orders.company                    3466
Orders.order_number                  0
Orders.reorder_id                 4112
Orders.external_source            3829
Orders.external_id                4189
Orders.currency                      0
Orders.sales_rep         

In [11]:
df["Products.id"].nunique()

1710

In [12]:
df["Products.unit"].unique()

array(['EA', 'CS', nan, 'PK', 'BG', 'PR', 'BX', 'BT', 'CV', 'DZ', 'ST',
       'CASE', 'RL', 'SET', 'GR', 'BOX', 'KT', 'PAIR', 'TB', 'PKG', 'CN'],
      dtype=object)

In [13]:
df["Customers.customer_type"].unique()

array([ 0.,  1., nan,  2.])

In [14]:
df["Order_Items.id"].unique()

array([ 5284,    31, 11655, ..., 20191, 20196, 20199], dtype=int64)

In [15]:
df["Products.hcpcs"].unique()

array(['E0149 +E0156', 'A4357',
       'NO GOVERNMENT OR SUPPLIER HCPCS CODE VERIFICATION', 'A9270',
       'E0730', nan,
       'NO CODE - PRODUCT DOES NOT MEET HCPCS SPECIFICATIONS', 'E0244',
       'NOT COVERED BY MEDICARE - CHECK REIMBURSEMENT WITH MEDICAID',
       'E1038', 'A6250', 'E0191', 'E0163', 'E0114',
       'SEE WWW.DMEPDAC.COM - CHECK "MISC" KEYWORD CODE OPTIONS', 'E0570',
       'A4927', 'E0245', 'A5120 ORA6250', 'A6257', 'L4387', 'A6234',
       'A4259', 'E0143;E0156', 'A6402', 'E0247', 'E0135', 'A4245',
       'E0143 +E0156', 'A4328', 'E0143', 'E0271', 'K0004 +E2201', 'E0243',
       'K0006 +E2201', 'K0001 ORK0002', 'L3999', 'E0248',
       'K0003 +E0966 +E1225 +K0195 +E0971', 'E0149', 'A6216', 'E0325',
       'B4034 ORB4035 ORB4036', 'E0971', 'K0821', 'E0168', 'K0001 +E2201',
       'A4349', 'L0625', 'E0950', 'E0105', 'E0274', 'L3908', 'E0181',
       'A4358', 'K0004 ORE1038 +K0195 ORE0990', 'K0001;K0002;E2201',
       'K0800', 'E0165', 'E0154', 'A4402', 'E0155', 'E0

In [16]:
df["Products.shopping_brand"].unique()

array(['MEDLINE', 'ROSCOE MEDICAL INC', nan, 'TELEFLEX MEDICAL',
       'LINKS MEDICAL PRODUCTS', 'FABRICATION ENTERPRISES INC',
       'SILARX PHARM', 'OMI INDUSTRIES', 'GEMINI PHARMACEUTICALS',
       'Drive Medical', 'GERITREX CORPORATION', 'SANDOZ', 'PDI, INC',
       'BASIC MEDICAL INDUSTRIES INC', 'HARTMANN-CONCO INC',
       'ROCHESTER MEDICAL CORP', 'OSSUR AMERICAS INC',
       'CONSOLIDATED MIDLAND CORPORATION', 'Lifestyle Essentials',
       'CYPRESS PHARMACEUTICAL INC', 'Mason Medical',
       'GERI-CARE PHARMACEUTICALS', 'Wenzelite', 'ITA-MED',
       'LIQUID CRYSTAL RESOURCES', 'Medquip', 'Purell &reg;',
       'MAGNO-HUMPHRIES LABORATORIES INC', 'WESTMED INC',
       'SANCTUARY HEALTH SDN BHD', 'NATIONAL NUTRITION INC',
       'POSEY COMPANY', 'FabLife&trade;', 'Dycem',
       'DEVON MEDICAL PRODUCTS', 'Fabrication Enterprises Inc.',
       'CanDo&reg;', 'GABRIALLA', 'Thera-Roll&reg;', 'WELCH-ALLYN',
       'Sup-R Band&reg;', 'ABBOTT DIABETES CARE', 'BIO CYBERNETICS',
   

In [17]:
df["Orders.payment_amount"].unique()

array([ 64.29,  29.99,  78.73, ...,  16.06, 989.1 ,  24.54])

In [18]:
# İhtiyacımız olan sutunlari aldik

In [19]:
dff=df[["Customers.id","Customers.customer_type","Customers.company","Orders.id","Order_Items.id","Order_Items.price","Products.id",
      "Products.price","Products.shopping_brand","Orders.subtotal","Orders.total","Products.name","Products.shopping_brand"]]

In [20]:
dff.head()

,Customers.id,Customers.customer_type,Customers.company,Orders.id,Order_Items.id,Order_Items.price,Products.id,Products.price,Products.shopping_brand,Orders.subtotal,Orders.total,Products.name,Products.shopping_brand
0,797,0.0,Company0,3758,5284,57.20,2310.0,57.64,MEDLINE,57.20,64.29,"Basic Steel Rollators,Green",MEDLINE
1,3,0.0,Company1,23,31,5.00,177.0,10.29,MEDLINE,20.00,29.99,Urinary Drain Bags,MEDLINE
2,3,0.0,Company1,9531,11655,68.78,1.0,68.78,MEDLINE,68.78,78.73,"SensiCare Nitrile Exam Gloves,Blue,XX-Large",MEDLINE
3,4,0.0,NaN,29,1816,19.56,983.0,19.56,MEDLINE,19.56,29.55,Basket for 2-Button Walkers,MEDLINE
4,5,0.0,Company3,30,40,36.05,991.0,36.05,ROSCOE MEDICAL INC,36.05,46.00,TENS 3000 Analog Unit,ROSCOE MEDICAL INC


In [21]:
dff["Customers.customer_type"] = dff["Customers.customer_type"].fillna(dff["Customers.customer_type"].mode()[0])

#### Maksimum satın alım yapan şirket?

In [23]:
# Şirketlere göre gruplama ve toplam sipariş miktarını bulma
total_purchase = df.groupby("Customers.company")["Orders.total"].sum()

# En yüksek toplam sipariş miktarına sahip şirketin adını bulma
max_purchase_company = total_purchase.idxmax()

print(f"En yüksek toplam satın alma yapan şirket: {max_purchase_company}")

En yüksek toplam satın alma yapan şirket: Company59


In [24]:
# Ürünlere göre gruplama ve en çok satialn ürünü bulma
toplam_urun_satis = df.groupby("Products.name")["Orders.total"].sum()

# Maksimum sipariş toplamına sahip şirketin adını bulma
en_cok_para_getiren_urun = toplam_urun_satis.idxmax()

print(f"En_cok_para_getiren_urun: {en_cok_para_getiren_urun}")

En_cok_para_getiren_urun: MoliCare Disposable Super Plus Briefs,27"-47"


In [25]:
ksk=pd.DataFrame(toplam_urun_satis)

In [26]:
ksk=ksk.rename(columns={"Orders.total": "toplam_getiriler"})

In [27]:
ksk

,toplam_getiriler
Products.name,
1 Box Of Medline Commode Liners with Absorbent Pad,37.98
"1-Ply Blockade AngelStat Surgical Gowns,Ceil Blue,Extra Large",235.05
"1/2"" self-adhesive hook material, 25 yard dispenser box, white",55.81
"1/2"" self-adhesive loop material, 25 yard dispenser box, white",55.81
"100% Cotton Unisex Reversible Hyperbaric Scrub Tops,Misty Green,MZS",134.98
...,...
Youth-Sized Walker Platform Attachment,61.66
"Zip-Style Biohazard Specimen Bags,Clear",58.36
iWalker Euro Style Rollator,169.99


In [28]:
dff= pd.merge(dff, ksk, on='Products.name')

In [29]:
dff.head()

,Customers.id,Customers.customer_type,Customers.company,Orders.id,Order_Items.id,Order_Items.price,Products.id,Products.price,Products.shopping_brand,Orders.subtotal,Orders.total,Products.name,Products.shopping_brand,toplam_getiriler
0,797,0.0,Company0,3758,5284,57.20,2310.0,57.64,MEDLINE,57.20,64.29,"Basic Steel Rollators,Green",MEDLINE,274.92
1,3,0.0,Company1,23,31,5.00,177.0,10.29,MEDLINE,20.00,29.99,Urinary Drain Bags,MEDLINE,154.20
2,3,0.0,Company1,9531,11655,68.78,1.0,68.78,MEDLINE,68.78,78.73,"SensiCare Nitrile Exam Gloves,Blue,XX-Large",MEDLINE,78.73
3,4,0.0,NaN,29,1816,19.56,983.0,19.56,MEDLINE,19.56,29.55,Basket for 2-Button Walkers,MEDLINE,265.13
4,5,0.0,Company3,30,40,36.05,991.0,36.05,ROSCOE MEDICAL INC,36.05,46.00,TENS 3000 Analog Unit,ROSCOE MEDICAL INC,46.00


In [30]:
dff["satis_adedi"]=dff["toplam_getiriler"]/dff["Products.price"]

In [31]:
dff.head()

,Customers.id,Customers.customer_type,Customers.company,Orders.id,Order_Items.id,Order_Items.price,Products.id,Products.price,Products.shopping_brand,Orders.subtotal,Orders.total,Products.name,Products.shopping_brand,toplam_getiriler,satis_adedi
0,797,0.0,Company0,3758,5284,57.20,2310.0,57.64,MEDLINE,57.20,64.29,"Basic Steel Rollators,Green",MEDLINE,274.92,4.769604
1,3,0.0,Company1,23,31,5.00,177.0,10.29,MEDLINE,20.00,29.99,Urinary Drain Bags,MEDLINE,154.20,14.985423
2,3,0.0,Company1,9531,11655,68.78,1.0,68.78,MEDLINE,68.78,78.73,"SensiCare Nitrile Exam Gloves,Blue,XX-Large",MEDLINE,78.73,1.144664
3,4,0.0,NaN,29,1816,19.56,983.0,19.56,MEDLINE,19.56,29.55,Basket for 2-Button Walkers,MEDLINE,265.13,13.554703
4,5,0.0,Company3,30,40,36.05,991.0,36.05,ROSCOE MEDICAL INC,36.05,46.00,TENS 3000 Analog Unit,ROSCOE MEDICAL INC,46.00,1.276006


In [32]:
jjj=pd.DataFrame(dff[["Products.id","satis_adedi"]])

In [33]:
jjj=jjj.drop_duplicates()

#### En cok satilan urun ?

In [35]:
# En çok satılan ürünün bulma
en_cok_satilan_urun_indexi = dff["satis_adedi"].idxmax()
en_cok_satilan_urun = dff.loc[en_cok_satilan_urun_indexi]

print(f"{en_cok_satilan_urun}")

Customers.id                                              213
Customers.customer_type                                   0.0
Customers.company                                         NaN
Orders.id                                                 941
Order_Items.id                                           1686
Order_Items.price                                        8.12
Products.id                                            1863.0
Products.price                                          11.17
Products.shopping_brand                               MEDLINE
Orders.subtotal                                        154.58
Orders.total                                           154.58
Products.name              Silent Knight Pill Crusher Pouches
Products.shopping_brand                               MEDLINE
toplam_getiriler                                      5476.39
satis_adedi                                        490.276634
Name: 200, dtype: object


In [44]:
sira=dff.sort_values(("satis_adedi"),ascending=False)

In [46]:
sira=sira[["Products.id","Products.name","satis_adedi"]]

In [48]:
sira.reset_index(inplace=True)

In [50]:
sira_unique = sira.drop_duplicates(subset=['Products.name'])

#### En çok satılan ürünlerin sıralamasi

In [53]:
sira_unique.head()

,index,Products.id,Products.name,satis_adedi
0,3269,1863.0,Silent Knight Pill Crusher Pouches,490.276634
6,1508,1846.0,"Disposable Emergency Blanket,Not Applicable",388.389483
24,3450,789.0,"Phlebotomy Sharps Containers,Red,1.000 QT",336.377622
29,3117,8385.0,Corrugated Aerosol Tubing,259.346405
30,1703,2110.0,"MoliCare Disposable Super Plus Briefs,39""-59""",246.837674


### Matrix Factorization

In [64]:
products_matrix=dff.pivot_table(index=["Customers.id"],columns=["Products.name"],values="Orders.subtotal")

In [65]:
products_matrix.head()

Products.name,1 Box Of Medline Commode Liners with Absorbent Pad,"1-Ply Blockade AngelStat Surgical Gowns,Ceil Blue,Extra Large","1/2"" self-adhesive hook material, 25 yard dispenser box, white","1/2"" self-adhesive loop material, 25 yard dispenser box, white","100% Cotton Unisex Reversible Hyperbaric Scrub Tops,Misty Green,MZS","100% Cotton Unisex Reversible Tops,Ceil Blue,MHS","18"" Hamper Stands,Standard","2"" self-adhesive loop material, 10 yard dispenser box, tan","2"" self-adhesive loop material, 25 yard dispenser box, tan","2-Ply Tissue/Poly Professional Towels,Not Applicable",2000 Wheelchairs,"24 Hour Urine Collection Bottle,3000.0 ML","3 Pack, Hospital Receiving Blankets, Baby Blankets, 100% Cotton, 30x40, Elephant","3 Pack, Hospital Receiving Blankets, Baby Blankets, 100% Cotton, 34x40, Footprints",3 Wheel Walker Rollator with Basket Tray and Pouch,"3"" Wheels For Walkers","3/4"" Guardian Utility Tips,Gray","5"" Wheels For Walkers",6 Cylinder Oxygen Carrier,6-spring hand grip (each),"8"" Rear Walker Wheel Attachment","A&D Ointment by H & H Laboratories Inc,384.00 ML",Accu-Therm Hot/Cold Gel Packs,Accu-Therm Insulated Hot Pack,Accu-Therm Non-Insulated Hot Pack,"Accutouch Chemo Nitrile Exam Gloves,Blue,Medium","Active Liquid Protein Nutritional Supplement,30.000 OZ",AcuForce&reg; 7.0 Massage Tool,"Addipak Sterile Saline Solution,3.00 ML",Adjustable Angle Rotating Suction Cup Grab Bar,Adjustable Height Bathtub Grab Bar Safety Rail,Adjustable Height Home Bed Assist Handle,"Adjustable Height Rollator with 6"" Wheels",Adjustable Height Teak Bath Bench Stool,Adjustable Lightweight Folding Cane with Gel Hand Grip,Adult Bariatric Crutches,"AdvanTrode&reg; Elite Electrode, 2"" round, tan tricot, 40/box","Advantage Magnetic Patient Alarms,Yellow",Advantage Therapeutic Homecare Foam Mattress,Aero-Pulse Pressure Pads,Aeroflow II Wheelchair Cushions,Aeromist Plus Nebulizer Compressor with Disposable Nebulizer Kit,All Purpose Rolling Shopping Utility Cart,"Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves,Large","Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves,Medium","Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves,Small","Aloetouch 3G Powder-Free Latex-Free Synthetic Exam Gloves,X-Large","Aloetouch Extended Cuff Chemo Nitrile Exam Gloves,Green,Medium","Aloetouch Extended Cuff Chemo Nitrile Exam Gloves,Green,Small","Aloetouch Extended Cuff Chemo Nitrile Exam Gloves,Green,X-Large",...,Walker Basket,Walker Basket Insert,Walker Carry Pouch/Tote,Walker Cup & Cane Holder,Walker Economy Basket with Plastic Tray,Walker Flip Tray,Walker Glide Ski,Walker Platform Attachment,Walker Rear Tennis Ball Glides,"Walker Rollator with 6"" Wheels, Fold Up Removable Back Support, and Padded Seat",Walker Ski Glides,Walker Ski/Glider,Walking Crutches with Underarm Pad and Handgrip,"Wall mat hanger, 86"" L x 8"" H x 16"" D, 10 capacity",Wallaby Headrest Extension,Wallaby Pediatric Folding Wheelchair,"Washable Cotton Material Gait Belts,Black","Washable Cotton Material Gait Belts,Red, White & Blue Stripes","Washable Cotton Material Gait Belts,Yellow","Water Soluble Bags,Clear",Waterproof Cast Protector,WaxWel&reg; Paraffin Bath - Accessory Package - 6 Terry Hand Mitts ONLY,Wedge Cushion with Stretch Cover,Wheelchair Anti-Fold/Anti-Theft Devices,Wheelchair Carry Pouch for Oxygen Cylinders,Wheelchair Drainage Bag Holders,Wheelchair Elevating Legrest Assembly Part,Wheelchair IV Pole Attachments,"Wheelchair Oxygen Tank Holder by Maddak,Black",Wheelchair Rear Anti-Tip Devices,Wheelchair Seat Belt,"Wheelchair tray with rim and straps, 24"" x 20"", wood finish","Wheelchair tray, economy, 24"" x 20"", wood finish","Whisper Ultra Quiet Bath Lift, Blue",Winnie Deluxe 3 Wheel Rollator Walker,Winnie Lite Supreme 3 Wheel Walker Rollator,Winnie Wagon All Purpose Shopping Utility Cart,"Wooden Applicator Sticks,6.00 IN",Wound Measuring Bullseye,Woven Sterile Gauze Sponges,"Wrist and Forearm Splint with Abducted Thumb,Large","Wrist and Forearm

In [67]:
# Ürünlerin siparişlerinin miktarları

In [84]:
urun=products_matrix['Silent Knight Pill Crusher Pouches']

In [86]:
urun.value_counts()

Silent Knight Pill Crusher Pouches
37.99     14
112.87     5
114.07     4
117.31     3
112.98     2
45.80      2
80.98      2
75.98      2
486.43     1
107.98     1
150.86     1
122.88     1
465.90     1
91.60      1
154.58     1
112.73     1
86.63      1
382.80     1
141.37     1
11.98      1
31.90      1
81.40      1
134.20     1
108.98     1
Name: count, dtype: int64

In [88]:
products_matrix.corrwith(urun)

Products.name
1 Box Of Medline Commode Liners with Absorbent Pad                                     NaN
1-Ply Blockade AngelStat Surgical Gowns,Ceil Blue,Extra Large                          NaN
1/2" self-adhesive hook material, 25 yard dispenser box, white                         NaN
1/2" self-adhesive loop material, 25 yard dispenser box, white                         NaN
100% Cotton Unisex Reversible Hyperbaric Scrub Tops,Misty Green,MZS                    NaN
                                                                                        ..
Youth-Sized Walker Platform Attachment                                                 NaN
Zip-Style Biohazard Specimen Bags,Clear                                                NaN
iWalker Euro Style Rollator                                                            NaN
sEMG - MyoTrac Home Trainer with Myoscan Active Sensor and extender cable, 1-channel   NaN
sEMG - single use uni-gel electrodes only, case of 100                      

In [89]:
similar_product=products_matrix.corrwith(urun)

In [92]:
similar_product=similar_product.dropna() #dropna boş verileri siliyor

In [94]:
similar_product.sort_values(ascending=False)

Products.name
DenTips Oral Swabsticks,Blue                     1.000000
Silent Knight Pill Crusher Pouches               1.000000
Silent Knight Pill Crushers                      0.995221
Non-Sterile Graduated Plastic Medicine Cups,1    0.670950
dtype: float64

### KNN

In [96]:
csns=pd.DataFrame()

In [98]:
csns["Products.id"]=df["Products.id"]

In [100]:
csns["Products.unit"]=df["Products.unit"]

In [102]:
csns["Products.name"]=df["Products.name"]

In [104]:
csns=csns.drop_duplicates()

In [106]:
csns

,Products.id,Products.unit,Products.name
0,2310.0,EA,"Basic Steel Rollators,Green"
1,177.0,EA,Urinary Drain Bags
2,1.0,CS,"SensiCare Nitrile Exam Gloves,Blue,XX-Large"
3,983.0,EA,Basket for 2-Button Walkers
4,991.0,EA,TENS 3000 Analog Unit
...,...,...,...
4173,14974.0,EA,"CanDo&reg; Soft Pliable Medicine Ball - 5"" Dia..."
4176,16660.0,EA,"CanDo&reg; Foam Roller - White PE foam - 6"" x ..."
4187,15649.0,EA,Relief Pak&reg; HotSpot&reg; Moist Heat Pack -...
4191,2707.0,EA,Spitfire EX Compact Travel Power Mobility Scoo...


In [109]:
csns_matrix=csns.pivot_table(index='Products.id', columns=csns["Products.unit"].values, values='Products.unit', aggfunc='count', fill_value=0)

In [111]:
csns_matrix

,BG,BOX,BT,BX,CASE,CN,CS,CV,DZ,EA,GR,KT,PAIR,PK,PKG,PR,RL,SET,ST,TB
Products.id,,,,,,,,,,,,,,,,,,,,
1.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
19.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
20.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
22.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
30.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24644.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
24699.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
24824.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [123]:
kos=pd.merge(csns,csns_matrix, on="Products.id")

In [127]:
kos=kos=pd.merge(kos,jjj, on="Products.id")

In [129]:
kos.shape

(1670, 24)

In [119]:
kos

,Products.id,Products.unit,Products.name,BG,BOX,BT,BX,CASE,CN,CS,CV,DZ,EA,GR,KT,PAIR,PK,PKG,PR,RL,SET,ST,TB,satis_adedi
0,2310.0,EA,"Basic Steel Rollators,Green",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4.769604
1,177.0,EA,Urinary Drain Bags,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,14.985423
2,1.0,CS,"SensiCare Nitrile Exam Gloves,Blue,XX-Large",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.144664
3,983.0,EA,Basket for 2-Button Walkers,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,13.554703
4,991.0,EA,TENS 3000 Analog Unit,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1.276006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1665,14974.0,EA,"CanDo&reg; Soft Pliable Medicine Ball - 5"" Dia...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1.534946
1666,16660.0,EA,"CanDo&reg; Foam Roller - White PE foam - 6"" x ...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.986175
1667,15649.0,EA,Relief Pak&reg; HotSpot&reg; Moist Heat Pack -...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1.689628
1668,2707.0,EA,Spitfire EX Compact Travel Power Mobility Scoo...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.900000


In [131]:
product_dict = {}

for index, row in kos.iterrows():
    id = row['Products.id']  # Ürün kimliği
    name = row['Products.name']  # Ürün kimliği
    unit_values = [row[col] for col in kos.columns[4:]]  # 4. sütundan sonraki değerler
    adet=row["satis_adedi"]
    product_dict[id]=(id,adet,name,unit_values)


In [132]:
product_dict

{2310.0: (2310.0,
  4.769604441360166,
  'Basic Steel Rollators,Green',
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   4.769604441360166]),
 177.0: (177.0,
  14.98542274052478,
  'Urinary Drain Bags',
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   14.98542274052478]),
 1.0: (1.0,
  1.1446641465542309,
  'SensiCare Nitrile Exam Gloves,Blue,XX-Large',
  [0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1.1446641465542309]),
 983.0: (983.0,
  13.554703476482619,
  'Basket for 2-Button Walkers',
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   13.554703476482619]),
 991.0: (991.0,
  1.2760055478502081,
  'TENS 3000 Analog Unit',
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0

In [134]:
from scipy import spatial

def compute_distance(a,b):
    unit_distance=spatial.distance.cosine(a[3],b[3])
    pop_distance=abs(a[1]-b[1])
    return unit_distance+pop_distance

In [135]:
product_dict[177.0]

(177.0,
 14.98542274052478,
 'Urinary Drain Bags',
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14.98542274052478])

In [136]:
product_dict[19.0]

(19.0,
 6.436978480456741,
 'Disposable Forehead Thermometer',
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6.436978480456741])

In [137]:
compute_distance(product_dict[2259.0],product_dict[79.0])# 2 vektör arasındaki mesafe ---Bu measfe ne kadar az olursa ürünler o kadar benzer oluyor

3.3427146506103833

In [138]:
mesafeler=[]
for prod1 in product_dict:
    for prod2 in product_dict:
        mesafeler.append(compute_distance(product_dict[prod1],product_dict[prod2]))

In [139]:
mesafeler

[0.0,
 10.2256071534786,
 3.8878739722594604,
 8.793933922335206,
 3.596683469514,
 9.68065763634884,
 3.7372069467212845,
 18.413305521060735,
 35.69948945766151,
 36.06286565243962,
 40.50207679522877,
 0.4744174450649413,
 20.27528368100275,
 23.95243785855702,
 44.51089338836707,
 47.77261516037442,
 58.00332464032992,
 2.29610916040771,
 1.797078121611492,
 15.249390197335163,
 0.2219405026187422,
 8.144748147892447,
 7.079906645733507,
 3.3651161382728523,
 3.823503208685882,
 4.34964717083481,
 1.7388298905755966,
 57.4521704709486,
 28.291995762991952,
 6.841019067930289,
 0.7753495466870285,
 2.3547468662315363,
 3.782228173754183,
 0.8629198674577174,
 12.436080181712779,
 1.305478569924563,
 15.289036675829415,
 56.43486754681161,
 0.2678186497217614,
 2.6469770294655532,
 0.3860381194638117,
 2.4361249484038865,
 75.8592413222775,
 19.32281806160973,
 1.238656624670527,
 2.399514603241776,
 2.847260991298491,
 0.2051899805604086,
 22.114343817237074,
 41.65277862490108,
 9.

Sonuc olarak;
Eğer müşteri hakkında hiç bilgimiz yoksa ona en cok satilan ürünlerden tavsiye edebiliriz.
Müşteri satın alım yaptıysa matrix factorization kullanarak müşterinin alabileceği ürünlerdn tavsiye edebiliriz 
Veya KNN le hesapladığımız benzer ürünlerden tavsiye edebiliriz